# Sentence 버전 #1 : 단어 시퀀스 

## 반복자
### 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키는, 인수를 받지 않는 \_\_next__() 메서드를 구현하는 객체.
### 파이썬 반복자는 \_\_iter__() 메서드도 구현하므로 반복형이기도 하다.

In [1]:
import re
import reprlib

RE_WORD = re.compile("\w+")

class Sentence :
    
    def __init__(self, text) :
        self.text = text
        self.words = RE_WORD.findall(text)
        
    # __iter__ 을 구현하지 않아도 __getitem__ 을 구현하면 인덱스가 0 부터 반복한다.
    def __getitem__(self, index) :
        return self.words[index]
    
    def __len__(self) :
        return len(self.words)
    
    def __repr__(self) :
        return f"Sentence({reprlib.repr(self.text)})"

In [2]:
s = Sentence('"The time has come", the Walrus said,')
s 

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s :
    print(word)
    

The
time
has
come
the
Walrus
said


# Sentence 버전 #2 : 고전적인 반복자
## 반복형과 반복자의 차이
<br>

## 반복형 : 호출할 때마다 반복자를 새로 생성하는 \_\_iter__() 메서드를 가지고 있다.
## 반복자 : 개별 항목을 반환하는 \_\_next__() 메서드와 self를 반환하는 \_\_iter__() 메서드를 가지고 있다.

In [4]:
import re
import reprlib

RE_WORD = re.compile("\w+")

# 반복형
class Sentence :
    
    def __init__(self, text) :
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self) :
        return f"Sentence({reprlib.repr(self.text)})"
    
    def __iter__(self) :
        return SentenceIterator(self.words)
    
# 반복자
class SentenceIterator :
    
    def __init__(self, words) :
        self.words = words
        self.index = 0
        
    def __next__(self) :
        try :
            word = self.words[self.index]
        except :
            raise StopIteration()
        self.index+=1
        return word
    
    def __iter__(self) :
        return self

In [5]:
s = Sentence('"The time has come", the Walrus said,')
s 

Sentence('"The time ha... Walrus said,')

# Sentence 버전 #3 : 제너레이터 함수

In [6]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence :
    def __init__(self, text) :
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self) :
        return f'Sentence({self.text})'
    
    def __iter__(self) :
        for word in self.words :
            yield word

In [7]:
sentence = Sentence("Lorem ipsum dolor sit amet.")
for word in sentence :
    print(word)

Lorem
ipsum
dolor
sit
amet


# Sentence 버전 #4 : 느긋한 구현

In [8]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence :
    def __init__(self, text) :
        self.text = text
        
    def __repr__(self) :
        return f'Sentence({self.text})'
    
    # words 를 미리 만들지 않고 호출될 경우 하나씩 찾는다. -> 메모리 절약
    def __iter__(self) :
        for match in RE_WORD.finditer(self.text) :
            yield match.group()

In [9]:
sentence = Sentence("Lorem ipsum dolor sit amet.")
for word in sentence :
    print(word)

Lorem
ipsum
dolor
sit
amet


# Sentence 버전 #5 : 제너레이터 표현식

In [10]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence :
    def __init__(self, text) :
        self.text = text
        
    def __repr__(self) :
        return f'Sentence({self.text})'
    
    # words 를 미리 만들지 않고 호출될 경우 하나씩 찾는다. -> 메모리 절약
    def __iter__(self) :
        return (match.group() for match in RE_WORD.finditer(self.text))

In [11]:
sentence = Sentence("Lorem ipsum dolor sit amet.")
for word in sentence :
    print(word)

Lorem
ipsum
dolor
sit
amet


# 예제 : 등차수열 제너레이터

In [12]:
class ArithmeticProgression :
    def __init__(self, begin, step, end=None) :
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self) :
        # 신기하게 형변환하기..
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        index = 0
        while forever or result < self.end :
            yield result
            
            # 오차 누적을 줄이기 위해 result 에 self.step 을 더해나가지 않는다.
            index += 1
            result = self.begin + self.step * index

In [13]:
from decimal import Decimal
from fractions import Fraction


ap = ArithmeticProgression(0,1,3)
print(list(ap))

ap = ArithmeticProgression(0,0.5,3)
print(list(ap))

ap = ArithmeticProgression(0,1/3,1)
print(list(ap))

ap = ArithmeticProgression(0, Fraction(1,3), 1)
print(list(ap))

ap = ArithmeticProgression(0, Decimal('.1'), 0.3)
print(list(ap))

for x in ArithmeticProgression(0, Decimal('.1')) :
    if x > 1 : break
    print(x, end=' ')

[0, 1, 2]
[0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
[0.0, 0.3333333333333333, 0.6666666666666666]
[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]
[Decimal('0'), Decimal('0.1'), Decimal('0.2')]
0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0 

# 필터링 제너레이터 함수 : 예시

In [23]:
import itertools
sample = [5,4,2,8,7,6,3,0,9,1]

list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [24]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [25]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [27]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [29]:
list(itertools.accumulate(range(1,11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

# 매핑 제너레이터 함수 : 예시

In [33]:
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [34]:
# 이항 연산자도 매핑이 가능하다.
import operator
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [36]:
# 매핑은 짧은 길이를 가진 반복자가 끝날 때까지 실행되는 제너레이터이다.
list(map(operator.mul, range(11), [2,4,8]))

[0, 4, 16]

In [38]:
import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [45]:
print(list(enumerate(itertools.accumulate(sample), 1)))
list(itertools.starmap(lambda a,b : b/a, enumerate(itertools.accumulate(sample), 1)))

[(1, 5), (2, 9), (3, 11), (4, 19), (5, 26), (6, 32), (7, 35), (8, 35), (9, 44), (10, 45)]


[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

# 병합 생성자 함수 : 예시

In [47]:
# 일반적으로 병합 생성자는 두개 이상의 반복자를 전달 받아 병합한다.
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [48]:
# 하나만 전달할수도 있지만 별 의미가 없다.
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [49]:
list(enumerate('ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [52]:
# from_iterable 은 반복자 안에 원소가 여러개일 경우 차례대로 병합한다.
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [53]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [54]:
list(zip('ABC', range(5), [10,20,30,40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [55]:
# 긴 반복자를 기준으로 병합할 수 도 있다.
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [56]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

# itertools.product() 제너레이터 함수 : 예시
### itertools.product() 는 데카르트 곱을 느긋하게 계산한다.

In [57]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [58]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product("AK", suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [59]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [61]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [65]:
# 0과 1로 만들 수 있는 3자리 중 가장 작은 것부터 순서대로 나열한 꼴이다.
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [68]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows : print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


# 항목 하나를 여러개로 확장하는 제너레이터 : 예시

In [74]:
list(itertools.islice(itertools.count(1, 0.3), 3))

[1, 1.3, 1.6]

In [76]:
cy = itertools.cycle('ABC')
list(itertools.islice(cy, 7))

['A', 'B', 'C', 'A', 'B', 'C', 'A']

In [78]:
list(itertools.repeat(8,4))

[8, 8, 8, 8]

In [80]:
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [81]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [83]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [85]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [88]:
# 중복순열이 된다.
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

# 재배치 제너레이터 : 예시

In [89]:
list(itertools.groupby('LLLLLAAGGG'))

[('L', <itertools._grouper at 0x2d8addeb8e0>),
 ('A', <itertools._grouper at 0x2d8adde8040>),
 ('G', <itertools._grouper at 0x2d8acc38910>)]

In [90]:
for char, group in itertools.groupby('LLLLLAAGGG') :
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L', 'L']
A -> ['A', 'A']
G -> ['G', 'G', 'G']


In [91]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat',
           'dolphin', 'shark', 'lion']
animals.sort(key = len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [92]:
for length, group in itertools.groupby(animals, len) :
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [93]:
for length, group in itertools.groupby(reversed(animals), len) :
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [110]:
# 제너레이터 복사

gen = (x for x in range(1,6))
# 둘은 같은 제너레이터 이다.
gen_copy = gen
next(gen)

print(list(gen))
print(list(gen_copy))

# g1,g2는 서로 독립적인 제너레이터이지만 gen에게는 종속적이다.
gen = (x for x in range(1,6))
g1, g2 = itertools.tee(gen)
# next(gen)
next(g1) 

print(list(g1))
print(list(g2))

[2, 3, 4, 5]
[]
[2, 3, 4, 5]
[1, 2, 3, 4, 5]
